# Optimization of TM model for ABA 2019 synphys data

### load MATLAB mat (=hdf) file with a structure

In [ ]:
import numpy as np
import pandas as pd
import scipy.io

# function to load table variable from MAT-file
def loadtablefrommat(matfilename, tablevarname, columnnamesvarname):
    """
    read a struct-ified table variable (and column names) from a MAT-file
    and return pandas.DataFrame object.
    """

    # load file
    mat = scipy.io.loadmat(matfilename)

    # get table (struct) variable
    tvar = mat.get(tablevarname)
    data_desc = mat.get(columnnamesvarname)
    types = tvar.dtype
    fieldnames = types.names

    # extract data (from table struct)
    data = None
    for idx in range(len(fieldnames)):
        if fieldnames[idx] == 'data':
            data = tvar[0][0][idx]
            break;

    # get number of columns and rows
    numcols = data.shape[1]
    numrows = data[0, 0].shape[0]

    # and get column headers as a list (array)
    data_cols = []
    for idx in range(numcols):
        data_cols.append(data_desc[0, idx][0])

    # create dict out of original table
    table_dict = {}
    for colidx in range(numcols):
        rowvals = []
        for rowidx in range(numrows):
            rowval = data[0,colidx][rowidx]
            if type(rowval[0]) == np.ndarray and rowval.size > 0:
                rowvals.append(rowval[0])
            else:
                rowvals.append(rowval)
        table_dict[data_cols[colidx]] = rowvals
    return pd.DataFrame(table_dict)

In [ ]:
do_normed_data= 1
if do_normed_data==0:
    df = loadtablefrommat('STP_cortex_exc_UDF3_struct.mat', 'table_struct', 'table_columns')
else:
    #df = loadtablefrommat('STP_cortex_exc_UDF3_struct_2.mat', 'table_struct', 'table_columns')
    #df = loadtablefrommat('STP_cortex_exc_UDF3_struct_3.mat', 'table_struct', 'table_columns') # fitted pulses medians for each cell
    df = loadtablefrommat('STP_cortex_exc_UDF4_struct.mat', 'table_struct', 'table_columns') # fitted pulses medians for each cell

df = df.iloc[1:,:]
df.head()

In [ ]:
df2 = pd.read_excel('aba_synapses_2.xlsx')
df2.head()

In [ ]:
def  STP_sim(x, T, init_state=None, model_type='tm5' ):
    # transform labels from TM to An:A1
    #f = 20 # Hz
    #N = 3
    #T = np.arange(N)*1000/f

    N    = len(T)
    #nc   = ge_data.shape[0]

    #x_lower = np.array([1,       0.01,    1,      0.1,     1])
    #x_upper = np.array([10000,   1,      10000,    10,     1])
    

    #stp_ns =          ['tF',    'p0',    'tD',   'dp/p0', 'A']
    #for jj in range(len(stp_ns)):
    #    nsj = stp_ns[jj]
    #    ge_data.loc[:,nsj] = np.maximum(ge_data.loc[:,nsj].values,x_lower[jj])
    #    ge_data.loc[:,nsj] = np.minimum(ge_data.loc[:,nsj].values,x_upper[jj])
    #    if nsj=='dp/p0':
    #        p0   = ge_data.loc[:,'p0'].values
    #        dpp0 = ge_data.loc[:,'dp/p0'].values
    #        dp = p0*dpp0
    #        dp = np.minimum(dp,1)
    #        ge_data.loc[:,nsj] = dp/(p0 +np.finfo(float).eps)

    #dpp0 = ge_data.loc[:,'dp/p0'].values
    #p0   = ge_data.loc[:,'p0'].values
    #tF   = ge_data.loc[:,'tF'].values
    #tD   = ge_data.loc[:,'tD'].values
    #A    = ge_data.loc[:,'A'].values
    
    
    
    tF      = x[0].astype(float)
    p00     = x[1]
    tD      = x[2].astype(float)
    dp      = x[3]
    A       = x[4]
    
    #breakpoint()
    mod_fdr2=False
    if model_type=='tm5_fdr2':  # should be :check freq. dependent recovery
        tDmin     = x[5]
        dd        = x[6]
        t_FDR     = x[7]
        mod_fdr2=True
        tDmax  = tD
        itDmin = 1/tDmin
        itDmax = 1/tDmax
        #breakpoint()
        
    mod_smr=False
    if model_type=='tm5_smr':  # should be :check freq. dependent recovery
        t_SMR   = x[8]
        dp0     = x[9]
        mod_smr=True
        #p00  = p00


    #As = np.zeros((nc,N))
    #n = np.zeros((nc,))
    #p = np.zeros((nc,))
    #ns2 = np.zeros((nc,N))
    #ps2 = np.zeros((nc,N))
    As = np.zeros((N,))
    #n = np.zeros((1,))
    #p = np.zeros((1,))
    state = np.zeros((N,4))
    #ps2 = np.zeros((N,))
    #ds2 = np.zeros((N,))

   
    if init_state is None :
        n = 1
        p0=p00
        p = p0
        d = 0
    else:
        n = init_state[0]
        p = init_state[1]
        d = init_state[2]
        p0= init_state[3]

    #ns2[i]=n
    #ps2[i]=p
    #ds2[i]=d
    
    for i in range(0,N):
        if i==0:
            Dt = T[i]
        else:
            Dt = T[i]-T[i-1]
        #n = 1 - (1 - (n -p*n))*np.exp((-Dt/tD).astype(float))
        #p=p0 -(p0-(p + dpp0*p0*(1-p)))*np.exp((-Dt/tF).astype(float))
        
        if mod_fdr2:
            d0=d
            d = d*np.exp(-Dt/t_FDR) 
            n = 1 - (1 - n )*np.exp(-Dt*itDmax -(itDmin -itDmax)*t_FDR*(d0-d))
        else:
            n = 1 - (1 - n )*np.exp(-Dt/tD )
            
        if mod_smr:
            p01=p0
            p0=p00 + (p0 -p00)*np.exp(-Dt/t_SMR)
            p=p0 +(p -p01)*np.exp(-Dt/tF)
        else:
            p=p0 +(p -p0)*np.exp(-Dt/tF)
            
        
        #tD = 1/(itDmax + (itDmin -itDmax)*d)
        #n = 1 - (1 - (n -p*n))*np.exp((-Dt/tD))
        #p=p0 +(p + dp*(1-p)-p0)*np.exp((-Dt/tF))

        As[i]=A*n*p
       
        n = n*(1-p)
        p = p + dp*(1-p)
        if mod_fdr2:
            d  = d + dd*(1-d) 
        if mod_smr:
            p0  = p0 - dp0*p0    
        
        #ns2[i]=n
        #ps2[i]=p
        #ds2[i]=d 
        state[i] = [n,p,d,p0]
        #breakpoint()


    #aa = [As, ns2, ps2]
    
    #breakpoint()
    #return As, ns2, ps2, dpp0, p0, tF, tD, A
    return As, state

In [ ]:
x = [1.00000000e+01, 4.00000000e-01, 1.00000000e+02, 4.00000000e-01,
 4.59968078e+00, 1.00000000e+01,
 5.00000000e-04, 1.00000000e+02]
x = np.array(x)
T  = np.arange(8)*50
As, st = STP_sim(x, T, init_state=None )
As

In [ ]:
st[:,0]

In [ ]:
st[:,1]

In [ ]:
st[:,2]

In [ ]:
st[:,3]

In [ ]:
def Q_TM_aba_synphys(x, amps,sig,Ts,DT0,model_type='tm5'):
    #Q, A, A3 =  QA_TM_aba_synphys(x, amps,sig,Ts)
    Q, A =  QA_TM_aba_synphys(x, amps,sig,Ts,DT0,model_type)
    return Q    

In [ ]:
def QA_TM_aba_synphys(x, amps,sig,Ts,DT0,model_type='tm5'):
    #amps=args[0] 
    #sig=args[1] 
    #Ts=args[2]
    
    
    fl=0
    if fl==0:
        ams = x[4:7]
        #breakpoint()
        if len(x)>7:
            if model_type=='tm5_fdr2':
                x2=np.copy(np.delete(x,[5,6]))
            if model_type=='tm5_smr':
                x2=np.copy(np.delete(x,[5,6]))    
            
            #x2[5:] = x[7:]
        else:
            x2=np.copy(x[0:5])
    
    n  = [0, 32, 44]
    t2 = [6, 1,  1]
    #T2  =[125, 250, 500, 1000, 2000, 4000]
    eps = np.finfo(float).eps
    z4 = np.arange(4)
    z8 = np.arange(8)
    A = np.zeros((amps.shape[0],))
    #A3 = np.ones((amps.shape[0],))
    #S3 = np.zeros((amps.shape[0],))
    Q=0
    for ii in range(3):
        vv = z8 + n[ii]
        A1e = amps[vv]  
        S1e = sig[vv]
        if fl==0:
            x2[4] = ams[ii] # independent amplitudes for each stimulation frequency - to account for rundown etc.
        
        #vv_nonz = np.nonzero(np.abs(A1e[:])>eps*1e3)[0]
        vv_z = np.abs(A1e[:])<eps*1e3
        #print(vv_nonz)
        #if len(vv_nonz)>0:
        if sum(vv_z)!=len(A1e):
            # preconditioning:
            A1, states = STP_sim(x2, Ts[vv],model_type=model_type ) 
            np0 = states[-1]
            
            # first 8 stimuli responces:
            #A1, states = STP_sim(x2, Ts[vv]+DT0, init_state=np0, model_type=model_type ) 
            #breakpoint()
            A1[vv_z] = 0
            A[vv] = A1
            
            #Q = Q + np.sum((A1e[vv_nonz]-A1[vv_nonz])**2/(S1e[vv_nonz]**2 + eps))
            Q = Q + np.sum((A1e-A1)**2/(S1e**2 + eps))
            # recovery responces
            #np0 = [n12[-1], p12[-1], d12[-1]]
            np0 = states[-1]
            for i3 in range(t2[ii]):
                vv2 = vv[-1]+1+z4+4*i3
                A2e = amps[vv2] 
                S2e = sig[vv2] 
                ##A3[vv2] = np.median(A1e)
                #vv_nonz2 = np.nonzero(np.abs(A2e[:])>eps*1e3)[0]
                #print(vv_nonz2)
                vv_z2 = np.abs(A2e[:])<eps*1e3
                #if len(vv_nonz2)>0:
                if sum(vv_z2)!=len(A2e):
                    A2, states2 = STP_sim(x2, Ts[vv2], init_state=np0, model_type=model_type )
                    
                    A2[vv_z2] = 0
                    
                    A[vv2] = A2
                    
                    #breakpoint()
                    #A2 = A2/np.median(A1)
                    #A[vv2] = A2
                    #breakpoint()
                    
                    #Q = Q + np.sum((A2e[vv_nonz2]-A2[vv_nonz2])**2/(S2e[vv_nonz2]**2 + eps))
                    Q = Q + np.sum((A2e-A2)**2/(S2e**2 + eps))
    
    return Q, A #, A3

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
x = [1.00000000e+01, 4.00000000e-01, 1.00000000e+02, 4.00000000e-01,
 4.59968078e+00, 4.59968078e+00,4.59968078e+00, 1.00000000e+01,
 5.00000000e-04, 1.00000000e+02]

Ts = df.loc[1,'comments1']
Ts = np.array(Ts[0][:].strip().split()).astype(float)
print(Ts)

vv=np.arange(len(As))
npar=56
amps = np.ones(npar)
amps[vv]=As*1.05
sig = amps/5
DT0=2500000
model_type = 'tm5'
Q2,As2 = QA_TM_aba_synphys(x[0:7], amps,sig,Ts,DT0,model_type )
plt.plot(As2)
As2

In [ ]:
#i=1
def fit_TM_cell(i,df,par):
    #breakpoint()
    aba_case=False
    if 'aba_case' in par:
        aba_case = par['aba_case']
        
    model_type='tm5'
    if 'model_type' in par:
        model_type = par['model_type']   
        
    dont_plot=0
    if 'dont_plot' in par:
        dont_plot = par['dont_plot'] 
    
    if par['fited_pulses']==True:
        ampsa = df.loc[i,'stp_mean_fit_bootstrap']
        siga = df.loc[i,'stp_sigma_fit_bootstrap']  
        #if abs(ampsa[0])<1e-14:
            #ampsa = df.loc[i,'stp_mean_bootstrap']
            #siga = df.loc[i,'stp_sigma_bootstrap']
    else:
        ampsa = df.loc[i,'stp_mean_bootstrap']
        siga = df.loc[i,'stp_sigma_bootstrap']
        
    Ts = df.loc[i,'comments1']
    Ts = np.array(Ts[0][:].strip().split()).astype(float)
    
    #breakpoint()

    #A3 = df.loc[i,'comments']
    #A3i = np.array(A3[0][:].strip().split()).astype(float)
    #A4 = np.ones((len(A3),len(A3i)))
    #for ia3 in range(len(A3)):
    #    A3i = np.array(A3[ia3][:].strip().split()).astype(float)
    #    A4[ia3,:] = A3i
    ##A4.shape 
    
    
    ibs  = par['ibs']
    npar = 56
    #amps = ampsa[ibs*npar + np.arange(npar)]
    ampsa1 = ampsa.reshape((-1,npar))
    amps = ampsa1[ibs,:].ravel()
    
    if aba_case:
        # sig  = siga #[(ibs-1)*npar + np.arange(npar)]
        #A3e  = A4[:,ibs ].ravel()

        siga1 = siga.reshape((-1,npar))
        sig = siga1[ibs,:].ravel()
    else:
        #sig = np.std(ampsa1,axis=0).ravel() # reestimate sigma
        sig=pd.DataFrame(ampsa1).std(axis=0).values
        
    DT0 = 25000
    #breakpoint()
    # restrict set of fited data
    vv = par['amplitudes_selected_for_TM_training']
    #vv = np.arange(8)    # A 1:8
    #vv = np.arange(npar) # all
    #vv = np.concatenate([vv,vv+32, vv+44]) # without recovery
    
    amps0 = amps
    amps = np.zeros(amps.shape)
    amps[vv] = amps0[vv]

         #  tF   p0     tD  dp         A            A1           A2      tDmin  dd   t_FDR t_SMR dp0 
    x0 = [ 100,  0.1,  100, 0.1, amps[0]/0.1, amps[0]/0.1,  amps[0]/0.1, 10,   0.05, 100,  100,  0.02 ]
    
    x0 = np.array(x0)
    
    #dxx = np.array([1e3, 20, 1e3, 1e2,  20, 20, 20,  1e1, 20, 1e2 ])
    dxx = np.array([1e3,  10, 1e3, 10,  20, 20, 20,  1e3, 20, 1e3, 1e3, 50 ])
    
    #breakpoint()
    if model_type=='tm5':
        x0 = x0[0:7]
        dxx = dxx[0:7]
        
    if model_type=='tm4':
        x0 = x0[0:7]
        dxx = dxx[0:7]
        
    if model_type=='tm5_fdr2':
        x0 = x0[0:10]
        dxx = dxx[0:10]
        
    if model_type=='tm5_smr':
        x0 = x0    
        dxx = dxx
    #print(x0)
    Q0, A0 = QA_TM_aba_synphys(x0, amps, sig, Ts,DT0,model_type)
    #print(x0)
    print('initial Q: ',str(Q0))

    #           tF      p0    tD        dp           A                A1                  A2              tDmin   dd   tD2
    #xlower = [  0.1,   0.01,   0.1,     0.0001,     amps[0]/1*1e-1,   amps[0]/1*1e-1,     amps[0]/1*1e-1,  1,     1e-3, 1]
    xlower = x0/dxx
    xlower = np.array(xlower)
    #xupper = [  1e5,   1,      1e5,     1,          amps[0]/0.01*1e1, amps[0]/0.01*1e1,   amps[0]/0.01*1e1, 100,  1,    1e4]
    #xupper = np.array(xupper)
    xupper = x0*dxx
    bounds = [] #np.zeros(len(xlower))
    for ibo in range(len(xlower)):
        bounds = bounds + [(xlower[ibo],xupper[ibo])]
    tt=[]
    #import time
    #import scipy.optimize as opt
    t1 = time.time()
    
    #print(x0)
    #print(A0)
    #breakpoint()
    
    do_this=0
    if do_this==1:
        res1 = opt.shgo(Q_TM_aba_synphys, bounds , args=((amps, sig, Ts,DT0,model_type)), constraints=None, n=10,
                            iters=1, callback=None, minimizer_kwargs=None, options=None, sampling_method='simplicial')
        x_shgo = res1.x
    else:
        x_shgo = x0
    
    t2 = time.time()
    tt = tt + [t2-t1]
    print('shgo : elapsed time '+str(t2-t1)+'s ')

    t1 = time.time()
    do_good=True
    if do_good:
        popsize = 65 # best
        maxiter=1000
        tol=0.01
    else:
        popsize = 5  # fast
        maxiter=10
        tol=0.1
        
    #breakpoint()    
    do_this=0
    if do_this==1:
        res2 = opt.differential_evolution(Q_TM_aba_synphys, bounds , args=((amps, sig, Ts,DT0,model_type)), strategy='best1bin',
                           maxiter=maxiter, popsize=popsize, tol=tol, mutation=(0.5, 1), recombination=0.7, seed=None,
                           callback=None, disp=False, polish=True, init='latinhypercube', atol=0,
                           updating='immediate', workers=1)
        x_de = res2.x
    else:
        x_de = x0  
        
    t2 = time.time()
    tt = tt + [t2-t1]
    print('differential_evolution: elapsed time '+str(t2-t1)+'s ')

    t1 = time.time()
    do_good=False
    if do_good:
        maxiter=1000 # good
        accept=-5.0 
        maxfun=10000000.0
    else:
        maxiter=10 # fast
        maxfun=10.0
        accept=-1.0
    fl=0  
    do_this=0
    if do_this==1:
        res3 = opt.dual_annealing(Q_TM_aba_synphys, bounds , args=((amps, sig, Ts,DT0,model_type)), maxiter=1000, 
                              local_search_options={},
                              initial_temp=5230.0,restart_temp_ratio=2e-05, visit=2.62, 
                              accept=accept, maxfun=maxfun, seed=None,
                              no_local_search=False, callback=None, x0=None)
        x_da = res3.x
    else:
        x_da = x0 
        
    t2 = time.time()
    tt = tt + [t2-t1]
    print('dual_annealing: elapsed time '+str(t2-t1)+'s ')

    t1 = time.time()
    do_this=0
    if do_this==1:
        res4 = opt.basinhopping(Q_TM_aba_synphys, x0 , niter=1, T=1.0, stepsize=1.0,
                            minimizer_kwargs={'method':"L-BFGS-B", 'args':(amps, sig, Ts,DT0,model_type)}, 
                            take_step=None, accept_test=None, callback=None, interval=50, disp=False,
                           niter_success=None, seed=None)
        x_bh = res4.x
    else:
        x_bh = x0 
        
    t2 = time.time()
    tt = tt + [t2-t1]
    print('basinhopping: elapsed time '+str(t2-t1)+'s ')
    #t1 = time.time()
    #res4 = opt.basinhopping(Q_TM_aba_synphys, x0 , niter=100, T=1.0, stepsize=1.0,
    #                        minimizer_kwargs={'method':"L-BFGS-B", 'args':(amps, sig, Ts,DT0)}, 
    #                        take_step=None, accept_test=None, callback=None, interval=50, disp=False,
    #                        niter_success=None, seed=None)
    t1 = time.time()
    do_this=1
    if do_this:
        prob = pg.problem(jit_rosenbrock2(amps.reshape((-1,1)),sig.reshape((-1,1)),Ts,model_type,
                                          np.array(xlower).reshape((-1,)),np.array(xupper).reshape((-1,))))
        #pop = pg.population(prob=prob, size = 1000)
        uda = pg.de1220(gen = 100, allowed_variants = [2,3,7,10,13,14,15,16], variant_adptv = 1, ftol = 1e-6, xtol = 1e-6, memory = False)
        algo = pg.algorithm(uda)
        archi = pg.archipelago(n=32,algo=algo, prob=prob, pop_size=150)
        
        #pop2 = algo.evolve(pop)
        archi.evolve() 
        archi.wait()
        
        qq=np.array(archi.get_champions_f())
        xx=np.array(archi.get_champions_x())
        
        x_sade = np.exp(xx[qq.argmin(),:])

    else:
        x_sade  = x0 
    
    t2 = time.time()
    Q, A = QA_TM_aba_synphys(x_sade, amps, sig, Ts,DT0,model_type)
    print('self adaptive differential evolution: time ' + str(t2 - t1), '; Q ',str(Q)) 
    
    
    
    t1 = time.time()
    do_this=0
    if do_this==1:
        #res4 = opt.basinhopping(Q_TM_aba_synphys, x0 , niter=100, T=1.0, stepsize=1.0,
        #                        minimizer_kwargs={'method':"L-BFGS-B", 'args':(amps, sig, Ts,DT0)}, 
        #                        take_step=None, accept_test=None, callback=None, interval=50, disp=False,
        #                        niter_success=None, seed=None)
        #nst=50 # good
        nst = 10 #150 # better fit
        Qs =  np.zeros(nst)
        As =  np.zeros((nst,npar))
        #vx = np.arange(5)  # fit 1 amplitude
        vx = np.arange(len(x0)) # fit separate amplitude for each frequency
        nx = len(x0)
        xs =  np.zeros((nst,len(x0[vx]) ))
        for ist in range(nst):

            #x0i = np.exp(np.log(xlower) + np.random.rand(len(xlower))*(np.log(xupper)-np.log(xlower)))
            x0i = x0*np.exp( 2*(np.random.rand(len(xlower))-0.5)*np.log(dxx)  )
            x0i = x0i[vx]
            #x0i[4:7] = x0i[4]
            fl=1
            #breakpoint()
            resi=opt.minimize(Q_TM_aba_synphys, x0i, args=((amps, sig, Ts,DT0,model_type)), method=None, jac=None,
                      hess=None, hessp=None, bounds=bounds[0:nx],
                      constraints=(), tol=None, callback=None, 
                      options=None)
            x=resi.x
            Q,A = QA_TM_aba_synphys(x, amps, sig, Ts,DT0,model_type)
            As[ist,:] = A
            xs[ist,:] = x
            Qs[ist] = Q

            
            Q_rs = np.min(Qs)
            ia = np.nonzero(Q_rs==Qs)[0]
            x_rs=xs[ia,:].ravel()
            A_rs = As[ia,:].ravel()
    else: 
        x_rs=x0
        Q_rs = 1e5
        A_rs = np.zeros((npar,))
            
    t2=time.time()
    tt = tt + [t2-t1]
    print('random_start: elapsed time '+str(t2-t1)+'s ')
    Q2 = []
    algs = ['shgo', 'differential_evolution','dual_annealing','basinhopping', 'self ajusting differential evolution',  'random_starts' ]
    colors = ['y',   'm',                           'c',          'y',          'r',                                 'g' ]
        
    if dont_plot==0:
        #%matplotlib
        #import matplotlib.pyplot as plt
        A3e=1
        
        plt.errorbar(np.arange(len(amps)), amps*A3e, yerr=sig*A3e, fmt =  'bo')
        #plt.hold(True)


        ii=0
        x = x_shgo #res1.x
        Q, A = QA_TM_aba_synphys(x, amps, sig, Ts,DT0,model_type)
        Q2 = Q2 + [Q]
        #plt.plot(np.arange(len(amps)), A, 'o-' ,color=colors[ii])
        print(algs[ii]+ ' Q = '+str(Q)+' '+colors[ii])



        ii=1
        x = x_de #res2.x
        x2=x
        Q, A = QA_TM_aba_synphys(x, amps, sig, Ts,DT0,model_type)
        Q2 = Q2 + [Q]
        #plt.plot(np.arange(len(amps)), A, 'd-' ,color=colors[ii])
        print(algs[ii]+ ' Q = '+str(Q)+' '+colors[ii])
        ii=2
        x = x_da #res3.x
        #x3=x
        Q, A = QA_TM_aba_synphys(x, amps, sig, Ts,DT0,model_type)
        Q2 = Q2 + [Q]
        #plt.plot(np.arange(len(amps)), A, 'x--',color=colors[ii] )
        print(algs[ii]+ ' Q = '+str(Q)+' '+colors[ii])
        ii=3
        x = x_bh #res4.x
        Q, A = QA_TM_aba_synphys(x, amps, sig, Ts,DT0,model_type)
        Q2 = Q2 + [Q]
        #plt.plot(np.arange(len(amps)), A, '.--',color=colors[ii] )
        print(algs[ii]+ ' Q = '+str(Q)+' '+colors[ii])

        ii=4

        x = x_sade #res4.x
        Q, A = QA_TM_aba_synphys(x, amps, sig, Ts,DT0,model_type)
        Q2 = Q2 + [Q]
        plt.plot(np.arange(len(amps)), A, '.-',color='r')
        print(algs[ii]+ ' Q = '+str(Q)+' '+colors[ii])

        ii=5


        #Q, A = QA_TM_aba_synphys(x, amps, sig, Ts,DT0)
        Q=Q_rs 
        Q2 = Q2 + [Q]
        #plt.plot(np.arange(len(amps)), A_rs, '.--',color=colors[ii] )
        print(algs[ii]+ ' Q = '+str(Q)+' '+colors[ii])

        #algs_short = ['differential_evolution',  'random_starts' ]
        algs_short = ['random_starts' ]
        #plt.legend(algs)
        plt.legend(algs_short)

        #fig = plt.gcf()
        #set_position(plt.gca, [10, 10, 100, 500], which='both')
        plt.xlim((min(vv)-1,max(vv)+2)) 
        #thismanager = plt.get_current_fig_manager()
        #thismanager.window.SetPosition((500, 0))
        #plt.hold(False)
        plt.pause(0.5) 
    
    #fig = plt.gcf()
    xx = [x_sade,x_da,x_rs]
    return Q2, tt, xx, algs, colors

## STOP!!! - This will Fit all cells STP

In [ ]:
%matplotlib
import matplotlib.pyplot as plt
import time
import scipy.optimize as opt
#from pylab import *


t00 = time.time()
Q3 =[]
TT =[]
X = []
algs = ['shgo', 'differential_evolution','dual_annealing','basinhopping',  'random_starts' ]
colors = ['g',   'r',                           'c',          'y',         'b' ]
nalg = len(algs)

npar=56
vv = np.arange(8)    # A 1:8 20Hz
#vv = np.arange(npar) # all
#vv = np.concatenate([vv,vv+32, vv+44]) # without recovery
par = {'fited_pulses':True, 'amplitudes_selected_for_TM_training':vv}


for i in range(len(df.index)): #range(len(df.index)):
    print('\n\n'+str(i)+'\n')
    Ts = df.loc[i+1,'comments1']
    ampsa = df.loc[i+1,'stp_mean_fit_bootstrap']
    is_fited_pulses = (abs(ampsa[0])>1e-14)
    if (len(Ts)!=0)&((par['fited_pulses']==False)|is_fited_pulses):
    #try:
        #par['fited_pulses']=True
        Q2, tt, x, algs, colors = fit_TM_cell(i+1,df,par)
        nalg = len(tt)
    #except:
    else:
        print(Ts)
        Q2 = (10*np.zeros(nalg)).tolist()
        tt = (0*np.zeros(nalg)).tolist()
        x = (0*np.zeros(nalg)).tolist()
        
    Q3 = Q3 + [Q2]
    TT = TT + [tt]
    X = X + [x]
    
Q3 = pd.DataFrame(Q3)  
TT = pd.DataFrame(TT) 
t10 = time.time()
print('time : '+ str(t10-t00))

In [ ]:
Q3.columns = algs
algs2=algs
for ialg, alg in enumerate(algs2): 
    algs2[ialg] = alg + ' , time'
TT.columns = algs2
QT = pd.concat([Q3,TT],axis=1)

In [ ]:
X3 = pd.DataFrame(X)
X3.columns = ['differential_evolution best x', 'dual_annealing best x', 'random_starts best x', 'shgo best x', 'basinhopping best x']

QTX = pd.concat([QT,X3],axis=1)

QTX.to_hdf('fit_aba_2019_A1_8_fited_pulses_results','data')

In [ ]:
QT2 = QT.iloc[:,0:5].min(axis=1)
QT3 = (QT.iloc[:,0:5].divide(QT2, axis=0)).astype(float).mean()
print('Q, % from the best')
QT3

In [ ]:
print('Q, # wins')
(QT.iloc[:,0:5].divide(QT2, axis=0)==1).astype(float).sum()

In [ ]:
print('time, % from the de')
QT2 = QT.iloc[:,6]
QT4 = (QT.iloc[:,5:].divide(QT2, axis=0)).mean(axis=0)
QT4

In [ ]:
QT.mean(axis=0)

In [ ]:
QT2 = QT.iloc[:,0:5].min(axis=1)
QT3 = (QT.iloc[:,0:5].divide(QT2, axis=0)).astype(float).mean()
print('Q, % from the best')
QT3

In [ ]:
print('Q, # wins')
(QT.iloc[:,0:5].divide(QT2, axis=0)==1).astype(float).sum()

In [ ]:
print('time, % from the de')
QT2 = QT.iloc[:,6]
QT4 = (QT.iloc[:,5:].divide(QT2, axis=0)).mean(axis=0)
QT4

In [ ]:
QT.mean(axis=0)

In [ ]:
60*43*50/60/60

## START: fit STP for average synapse types

In [ ]:
df.head()

In [ ]:
#df3.columns

In [ ]:
alltypes = list(set(df.loc[:,'synapse_type_2'].astype(str)))
print(len(alltypes))
alltypes

In [ ]:
df2 = df2.set_index('Unnamed: 0')
df3 = df2.loc[0,:]
df3.head()

In [ ]:
df.head()

In [ ]:
df3.index = df.index
print(df3.shape)

In [ ]:
for i in df.index:
    st2 = df.loc[i,'synapse_type'][0]
    df.loc[i,'synapse_type'] = st2
    #st2 = df.loc[i,'synapse_type_2'][0]

In [ ]:
for i in df.index:
    st2 = df.loc[i,'synapse_type_2'][0]
    df.loc[i,'synapse_type_2'] = st2
    #st2 = df.loc[i,'synapse_type_2'][0]

In [ ]:
import re
syntype4 = df.loc[:,'synapse_type_2']
sel=np.ones(len(df.index))
for i in df.index:
    #st2 = df.loc[i,'synapse_type_2'][0]
    #df.loc[i,'synapse_type_2'] = st2
    st2 = df.loc[i,'synapse_type_2']
    
    #st3 = df.loc[i,'synapse_type'][0]
    #df.loc[i,'synapse_type'] = st3
    
    st3 = df3.loc[i,'synapse_type3']
    st21=re.split('\;',st2)
    st22=re.split('unknown',st2)
    if len(st22)>1:
        if (st22[0]=='')&(st3=='ex'):
            sel[i-1]=2
        else:
            sel[i-1]=0

In [ ]:
print(np.sum(sel==1))
print(np.sum(sel==2))
sel.shape

In [ ]:
df4 = df.loc[sel!=0,:]

In [ ]:
alltypes2 = list(set(df4.loc[:,'synapse_type_2'].astype(str)))
print(df4.shape)
print(len(alltypes2))
alltypes2

In [ ]:
#df5 = df4.set_index(['synapse_type','synapse_type_2']).sort_values('synapse_type')
df5 = df4.set_index(['synapse_type']).sort_values('synapse_type_2')
print(df5.shape)

In [ ]:
df5.head(10)

In [ ]:
df6 = df5.loc[:,'synapse_type_2'].value_counts()
df6.to_excel('numbers_of_synaptic_types.xlsx')
df6

In [ ]:
fits= df5.loc[:,'stp_mean_fit_bootstrap']
sel=np.ones(len(df5.index))
for i in range(df5.shape[0]):
    #st2 = df.loc[i,'synapse_type_2'][0]
    #df.loc[i,'synapse_type_2'] = st2
    #st2 = fits.iloc[i]
    if fits.iloc[i][0]==0:
        sel[i]=0
df7 = df5.loc[sel==1,:]        

In [ ]:
df8 = df7.loc[:,'synapse_type_2'].value_counts()
df8.to_excel('numbers_of_synaptic_types_with_fited_pulses.xlsx')
print((df8.loc[df8.loc[:]>4]).sum())
df8

In [ ]:
df7.columns

In [ ]:
#df9 = df7.loc[:,['synapse_type_2', 'stp_mean_fit_bootstrap']]
#stp_types = df9.groupby('synapse_type_2').mean()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

df7.index  = df7.loc[:,'synapse_type_2']


types = list(set(df7.loc[:,'synapse_type_2']))
df7accum = pd.DataFrame()
i = 2

df7i = df7.loc[df7.loc[:,'synapse_type_2'].isin([types[i]]),:]
fits = df7i.loc[:,'stp_mean_fit_bootstrap'].values
fits = pd.DataFrame(np.stack(fits,axis=1)).replace(to_replace=0, value=np.nan).T
fits = fits.divide(fits.iloc[:,0:8].median(axis=1), axis=0)
fitsnan = np.isnan(fits).astype(float)

fitsnums = df7i.loc[:,'numbers_of_currents'].values
fitsnums = pd.DataFrame(np.stack(fitsnums,axis=1)).T #.replace(to_replace=0, value=np.nan).T


fitsm = pd.DataFrame(fits.median(axis=0)).T 
fitss = pd.DataFrame(fits.std(axis=0)).T 

fitsnanm = fitsnan.mean(axis=0)    # if less than 1/2 cells have some protocole - dont' use it
#fitsm1 = fitsm
fitsm.loc[0,fitsnanm>0.5] = 0 #np.nan
npar = 56

plt.errorbar(np.arange(npar), fitsm.loc[:,0:npar-1].values.ravel(), yerr=fitss.loc[:,0:npar-1].values.ravel(), fmt =  'go')
plt.gca().set_title(types[i] +' , n='+str(fits.shape[0]))    
#fitsnums = df7i.loc[:,'numbers_of_currents'].values
#fn1 = np.array(fitsnums.loc[0,:][0].split()).astype(int)
#fitsnums = pd.DataFrame(np.stack(fitsnums,axis=1)).T #.replace(to_replace=0, value=np.nan).T
    

In [ ]:
# aggregate some synaptic types
fn1 = 'numbers_of_synaptic_types_with_fited_pulses_corrected.xlsx'
fn2 = 'numbers_of_synaptic_types_with_fited_pulses_corrected2.xlsx'
dfc1 = pd.read_excel(fn1)
dfc2 = pd.read_excel(fn2)
dfc1=dfc1.set_index('Unnamed: 0')
dfc2=dfc2.set_index('Unnamed: 0')

In [ ]:
dfc2.head()

In [ ]:
dfc1.head()

In [ ]:
types0 = list(set(df7.loc[:,'synapse_type_2']))
types = []
for i in range(len(types0)):
    ty0 = types0[i]
    typesi = [ty0]
    
    dfc1i = np.nonzero((dfc1.index==ty0))[0] + 2
    if len(dfc1i)>0:
        dfc1i2 = dfc1.loc[:,'Unnamed: 2']==dfc1i[0]
        if sum(dfc1i2)>0:
            typesi = typesi + dfc1.index[dfc1i2].tolist()
        
    dfc2i = np.nonzero((dfc2.index==ty0))[0][0] + 2
    dfc2i2 = dfc2.loc[:,'Unnamed: 2']==dfc2i
    if sum(dfc2i2)>0:
        typesi = typesi + dfc2.index[dfc2i2].tolist()    
        
    types = types + [typesi]    

In [ ]:
types

In [ ]:
#plt.plot(np.mean(fitss4.reshape((-1,npar)),axis=0  ))
#np.nonzero(fitss4.ravel()==0)[0]

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import scipy.stats

#types = list(set(df7.loc[:,'synapse_type_2']))
df7accum = pd.DataFrame()
Fitss2 = []
for i in  range(len(types)): #range(len(types)): #range(len(types)):
    df7i = df7.loc[df7.loc[:,'synapse_type_2'].isin(types[i]),:]
    fits = df7i.loc[:,'stp_mean_fit_bootstrap'].values
    fits = pd.DataFrame(np.stack(fits,axis=1)).replace(to_replace=0, value=np.nan).T
    ##fits = fits.divide(fits.iloc[:,0:8].median(axis=1), axis=0)
    fits = fits.divide(fits.iloc[:,0:8].median(axis=1), axis=0)

    if fits.shape[0]>=5:
        print(i)
        
        fitsnan = np.isnan(fits).astype(float)
        fitsnums = df7i.loc[:,'numbers_of_currents'].values
        fitsnums = pd.DataFrame(np.stack(fitsnums,axis=1)).T #.replace(to_replace=0, value=np.nan).T
                
        npar = 56
        
        do_bootstrap_here=1
        if do_bootstrap_here==1:
            nc = int(fits.shape[1]/npar)
            vv = np.arange(npar)
            fitsm1 = fits.copy()
            
            for c in range(1,nc):
                ibs = np.random.randint(0, high=fits.shape[0], size=fits.shape[0])
                cbs = np.random.randint(0, high=nc, size=fits.shape[0])
                for ib in range(len(ibs)):
                    
                    #fitsm.iloc[0,c*npar + vv ] = fits.iloc[ibs,c*npar + vv ].median(axis=0)
                    fitsm1.iloc[ib,c*npar + vv ] = fits.iloc[ibs[ib],cbs[ib]*npar + vv ].values #.median(axis=0)
            fitsm = pd.DataFrame(fitsm1.mean(axis=0)).T    
            fitss4 = pd.DataFrame(scipy.stats.sem(fitsm1.values, axis=0,  nan_policy='omit')).T
        else:
            fitsm = pd.DataFrame(fits.median(axis=0)).T 
         
        fitss5 = np.nanmean(fitss4.values.reshape((-1,npar)),axis=0  )
        fitss = pd.DataFrame(fits.std(axis=0)).T 

        fitsnanm = fitsnan.mean(axis=0)    # if less than 1/2 cells have some protocol data - dont' use it
        #fitsm1 = fitsm
        fitsm.loc[0,fitsnanm>0.3] = 0 #np.nan   
        
        fitsms = pd.DataFrame(np.reshape(fitsm.values, (-1,npar)))
        fitsss = pd.DataFrame(np.reshape(fitsm.values, (-1,npar)))
        fitsm2 =fitsms.mean(axis=0,skipna=True)
        
        do_bootstrap_sigma=True
        if do_bootstrap_sigma==False:
            fitss2 =fitss.loc[:,0:npar-1].values.ravel() #fitsss.mean(axis=0,skipna=True)
        else:
            fitss2 =fitsss.std(axis=0,skipna=True)
         
        # fill nans and set mean=0 if sigma==0 to not fit this points
        fitsm = fitsm.fillna(value=0)
        fitss2 = np.nan_to_num(fitss2)
        fitss4 = np.nan_to_num(fitss4.values)
        fitss5 = np.nan_to_num(fitss5)
        
        #iz = np.nonzero(fitss2==0)[0]
        #if len(iz)>0:
        #    fitsm.iloc[0,iz]=0  # only first bootstrap will be filled with zeros!!!
        iz = np.nonzero(fitss4.ravel()==0)[0]
        if len(iz)>0:
            fitsm.iloc[0,iz]=0  # only first bootstrap will be filled with zeros?
        
        #breakpoint()

        
        fitsn = pd.DataFrame(df7i.iloc[0,0:10]).T
        fitsm.index = fitsn.index
        fitss.index = fitss.index
        fitsmn = pd.concat([fitsn, fitsm ],axis=1)
        #df7accum = pd.concat([df7accum, fitsmn ],axis=0)

        df7accum = pd.concat([df7accum, fitsmn ],axis=0)

        

        #Fitss2 = Fitss2 + [fitss2]
        Fitss2 = Fitss2 + [fitss4]

        f, ax =plt.subplots(figsize=(15, 10))
        #f, ax = plt.figure()
        #ax = f.add_axes()
        #plt.title(stp_columns[i])
        #plt.title(types[i] +' , n='+str(fits.shape[0]))
        
        plt.plot(fitsm2,'bo-')
        plt.plot(fitsm.iloc[0,0:npar],'co-')
        plt.gca().set_title(str(types[i]) +' , n='+str(fits.shape[0]))
        plt.ylim((0,max(fitsm2)*1.05)) 
        #plt.hold(True)
        #fitss5  = np.mean(fitss4.reshape( (-1,npar)).T)
        plt.errorbar(np.arange(len(fitsm2)), fitsm2, yerr=fitss5, fmt =  'go')
        #plt.xticks(np.arange(len(yy1l.index)), yy1l.index, rotation=90)
        #plt.hold(False)
        #plt.pause(0.5) 
        
        df7accum.loc[df7accum.index[-1],'comments'] = str(types[i]) +' , n='+str(fits.shape[0])
        
        plt.pause(0.1)
df7accum.shape        

In [ ]:
### repack accumulated stp data for fitting

In [ ]:
df8 = pd.DataFrame([],columns = df.columns[11:])

for i in range(len(df7accum.index)):
    df10 = pd.DataFrame([[],[], [df7accum.iloc[i,10:].values],[Fitss2[i]]],index = df.columns[11:], columns = [df7accum.index[i]]).T
    df8 = pd.concat([df8,df10],axis=0)



df9 =pd.concat([df7accum.iloc[:,0:10],df8],axis=1)
df9

In [ ]:
df9.index[22]

## fit averaged stp

### Tsodyks Markram model 5 parameters + 3 parameters to fit changing depression recovery rate

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import time
import scipy.optimize as opt
from pygmo import 
#from pylab import *


t00 = time.time()
Q3 =[]
TT =[]
X = []
algs = ['shgo', 'differential_evolution','dual_annealing','basinhopping',  'random_starts' ]
colors = ['g',   'r',                           'c',          'y',         'b' ]
nalg = len(algs)

npar=56
#vv = np.arange(8)    # A 1:8 20Hz
vv = np.arange(npar) # all
vv = np.delete(vv,np.array([8,12,16,20,24,28,32,40,44,52,56])-1)
#vv = np.concatenate([vv,vv+32, vv+44]) # without recovery
par = {'fited_pulses':True, 'amplitudes_selected_for_TM_training':vv, 'aba_case':True, 'figure':0,
       'model_type' : 'tm5' }

par['model_type'] = 'tm5_fdr2'  # extended TM model

for i in  range(0,len(df9.index)): #range(22,23): #range(len(df9.index)):
    idx = df9.index[i]
    print('\n\n'+str(i)+'  '+df9.loc[idx,'comments']+'\n')
    Ts  = df9.loc[idx,'comments1']
    #ampsa = df.loc[idx,'stp_mean_fit_bootstrap']
    #is_fited_pulses = (abs(ampsa[0])>1e-14)
    
    is_fited_pulses = True
    if (len(Ts)!=0)&((par['fited_pulses']==False)|is_fited_pulses):
        fig, ax = plt.subplots(figsize=(20, 10))
        par['figure']=fig
    #try:
        #par['fited_pulses']=True
        Q2, tt, x, algs, colors = fit_TM_cell(idx,df9,par)
        nalg = len(tt)
    #except:
    else:
        print(Ts)
        Q2 = (10*np.zeros(nalg)).tolist()
        tt = (0*np.zeros(nalg)).tolist()
        x = (0*np.zeros(nalg)).tolist()
        
    Q3 = Q3 + [Q2]
    TT = TT + [tt]
    X = X + [x]
    

    
Q3 = pd.DataFrame(Q3)  
TT = pd.DataFrame(TT) 
t10 = time.time()
print('time : '+ str(t10-t00))


# Save results

Q3.columns = algs
algs2=algs
for ialg, alg in enumerate(algs2): 
    algs2[ialg] = alg + ' , time'
TT.columns = algs2
QT = pd.concat([Q3,TT],axis=1)

X3 = pd.DataFrame(X)
#X3.columns = ['differential_evolution best x', 'dual_annealing best x', 'random_starts best x', 'shgo best x', 'basinhopping best x']
X3.columns = ['differential_evolution best x', 'dual_annealing best x', 'random_starts best x']


QTX_extended = pd.concat([df9,QT,X3],axis=1)

QTX_extended.to_hdf('fit_aba_2019_A1_8_fited_pulses_TM_8_syntypes_results','data')

In [ ]:
from numba import jit, float64, int64

dummy_x = np.full((2000,), 1.)
import time

@jit(float64[:,:](float64[:,:],float64[:],float64[:,:],int64),nopython=True)
def  STP_sim2(x, T, init_state, model_type=1 ):
    N       = len(T)
    
    tF      = x[0,:]
    p00     = x[1,:]
    tD      = x[2,:]
    dp      = x[3,:]
    A       = x[4,:]
    
    N1      = len(tF)

    #breakpoint()
    mod_fdr2=False
    if model_type==2:  #tm5_fdr2 should be :check freq. dependent recovery
        tDmin     = x[5,:]
        dd        = x[6,:]
        t_FDR     = x[7,:]
        mod_fdr2=True
        tDmax  = tD
        itDmin = 1/tDmin
        itDmax = 1/tDmax
        #breakpoint()

    mod_smr=False
    if model_type==3:  #tm5+smr should be :check freq. dependent recovery
        t_SMR   = x[8,:]
        dp0     = x[9,:]
        mod_smr=True

    As = np.zeros((N,N1))
    #state = np.zeros((N1,4))

    n = np.full((N1,),1.0)
    p0=p00  #np.full((1,N1),p00) #p00
    p = p0 #p0
    d = np.full((N1,),0.0) #0
    if len(init_state)>0:
        n = init_state[0,:]
        p = init_state[1,:]
        d = init_state[2,:]
        p0= init_state[3,:]

    for i in range(0,N):
        if i==0:
            Dt = T[i]
        else:
            Dt = T[i]-T[i-1]

        if mod_fdr2:
            d0=d
            d = d*np.exp(-Dt/t_FDR) 
            n = 1 - (1 - n )*np.exp(-Dt*itDmax -(itDmin -itDmax)*t_FDR*(d0-d))
        else:
            n = 1 - (1 - n )*np.exp(-Dt/tD )

        if mod_smr:
            p01=p0
            p0=p00 + (p0 -p00)*np.exp(-Dt/t_SMR)
            p=p0 +(p -p01)*np.exp(-Dt/tF)
        else:
            p=p0 +(p -p0)*np.exp(-Dt/tF)

        As[i,:]=A*n*p

        n = n*(1-p)
        p = p + dp*(1-p)
        if mod_fdr2:
            d  = d + dd*(1-d) 
        if mod_smr:
            p0  = p0 - dp0*p0  
            
    state = np.concatenate((n,p,d,p0),axis=0).reshape((-1,4)).transpose() #[n,p,d,p0]
    state = np.concatenate((As,state),axis=0)

    return state


class jit_rosenbrock2:
    def fitness(self,x):
        amps = self.amps
        sig = self.sig
        Ts=self.Ts
        model_type = self.model_type
        N1 = int(x.size/self.dim)
        x = x.reshape((self.dim,N1))
        x1 = np.copy(np.exp(x))
        f = jit_rosenbrock2.QA_TM_aba_synphys(x1,amps,sig,Ts,model_type=model_type) 
        return f.reshape((-1,))
    
    def batch_fitness(self,dvs):
        amps = self.amps
        sig = self.sig
        Ts=self.Ts
        model_type = self.model_type
        x = dvs.reshape((self.dim,-1))
        x1 = np.copy(np.exp(x))
        dvf =  jit_rosenbrock2.QA_TM_aba_synphys(x,amps,sig,Ts,model_type=model_type) 
        
        return dvf.reshape((-1,))
    
    def has_batch_fitness(self):
        return True

    @jit(float64[:,:](float64[:,:],float64[:,:],float64[:,:],float64[:],int64),nopython=True)
    def QA_TM_aba_synphys(x,amps,sig,Ts,model_type=1): #QA_TM_aba_synphys
        fl=0
        if fl==0:
            #x = np.copy(x00).reshape((-1,1))
            
            ams = x[4:7,:]
            #breakpoint()
            if len(x)>7:
                if model_type==2: #'tm5_fdr2':
                    #x2=np.copy(np.delete(x,[5,6],axis=0))
                    x2=np.copy(np.concatenate((x[:5,:],x[7:,:]),axis=0))
                if model_type==3: #'tm5_smr':
                    #x2=np.copy(np.delete(x,[5,6],axis=0))
                    x2=np.copy(np.concatenate((x[:5,:],x[7:,:]),axis=0))

                #x2[5:] = x[7:]
            else:
                x2=np.copy(x[0:5,:])

        n  = [0, 32, 44]
        t2 = [6, 1,  1]
        #T2  =[125, 250, 500, 1000, 2000, 4000]
        eps = 1e-16 #np.finfo(float).eps
        
        N1=x.shape[1]
        N2=amps.shape[1]
        
        z4 = np.arange(4)
        z8 = np.arange(8)
        z0 = np.zeros((0,1))
        
        #A = np.zeros((amps.shape[0],1))
        Q=np.zeros((N2,N1))
        for ii in range(3):
            vv = z8 + n[ii]
            A1e = amps[vv,:]  
            S1e = sig[vv,:]
            if fl==0:
                x2[4,:] = ams[ii,:] # independent amplitudes for each stimulation frequency - to account for rundown etc.

            #vv_nonz = np.nonzero(np.abs(A1e[:])>eps*1e3)[0]
            vv_z = np.abs(A1e[:,0])<eps*1e3
            #print(vv_nonz)
            #if len(vv_nonz)>0:
            if np.sum(vv_z)!=len(A1e):
                # preconditioning:
                states = STP_sim2(x2, Ts[vv],z0,model_type=model_type ) 
                A1=states[:-4,:]
                np0 = states[-4:,:]
                #np0 = np0[1:]

                # first 8 stimuli responces:
                #A1, states = STP_sim2(x2, Ts[vv]+DT0, init_state=np0, model_type=model_type ) 
                A1[vv_z,:] = 0
                #A[vv,:] = A1

                #Q = Q + np.sum((A1e[vv_nonz]-A1[vv_nonz])**2/(S1e[vv_nonz]**2 + eps))
                Q[:] +=  np.sum((A1e-A1)**2/(S1e**2 + eps))
                # recovery responces
                #np0 = [n12[-1], p12[-1], d12[-1]]
                #np0 = states[-1]
                for i3 in range(t2[ii]):
                    vv2 = vv[-1]+1+z4+4*i3
                    A2e = amps[vv2,:] 
                    S2e = sig[vv2,:] 

                    vv_z2 = np.abs(A2e[:,0])<eps*1e3
                    #if len(vv_nonz2)>0:
                    if np.sum(vv_z2)!=len(A2e):
                        states2 = STP_sim2(x2, Ts[vv2], init_state=np0, model_type=model_type )
                        A2=states2[:-4,:]
                        A2[vv_z2,:] = 0

                        #A[vv2,:] = A2

                        #Q = Q + np.sum((A2e[vv_nonz2]-A2[vv_nonz2])**2/(S2e[vv_nonz2]**2 + eps))
                        Q[:] += np.sum((A2e-A2)**2/(S2e**2 + eps))

        return Q #, A #, A3

    #def get_nec(self):
    #    return 1

    #def get_nic(self):
    #    return 1

    def get_bounds(self):
        #return (np.full((self.dim,),-5.),np.full((self.dim,),10.))
        xl=np.full((self.dim,),0.)
        xu=np.full((self.dim,),1.)
        xl[:]=self.xlower
        xu[:]=self.xupper
        return (xl,xu)
    
    def gradient(self, x):
        return pg.estimate_gradient_h(lambda x: self.fitness(x), x)
    
    def __init__(self,amps,sig,Ts,model_type,xlower,xupper):
        self.dim = 12 #len(xlower) #xlower.shape[0]
        self.amps = amps 
        self.Ts = Ts
        self.sig = sig
        self.DT0 = float(0)
        self.model_type = 3 # model_type #'tm5'
        self.xlower = np.log(xlower)
        self.xupper = np.log(xupper)

In [ ]:
#np.std(ampsa1, axis=0 )

In [ ]:
########################## 1
t00 = time.time()
Q3 =[]
TT =[]
X = []
algs = ['shgo', 'differential_evolution','dual_annealing','basinhopping',  'random_starts' ]
colors = ['g',   'r',                           'c',          'y',         'b' ]
nalg = len(algs)

npar=56
vv = np.arange(7)    # A 1:8 20Hz


vv = np.arange(npar) # all
vv = np.delete(vv,np.array([8,12,16,20,24,28,32,40,44,52,56])-1)
##vv = np.concatenate([vv,vv+32, vv+44]) # without recovery
par = {'fited_pulses':True, 'amplitudes_selected_for_TM_training':vv, 'aba_case':True, 'figure':0,
       'model_type' : 'tm5','ibs':0 }

par['model_type'] = 'tm5_smr'  # extended TM model
#par['model_type'] = 'tm5_smr'



########################## 2
i0=6
idx = df9.index[i0]
print('\n\n'+str(i)+'  '+df9.loc[idx,'comments']+'\n')
Ts  = df9.loc[idx,'comments1']
#ampsa = df.loc[idx,'stp_mean_fit_bootstrap']
#is_fited_pulses = (abs(ampsa[0])>1e-14)

is_fited_pulses = True
if (len(Ts)!=0)&((par['fited_pulses']==False)|is_fited_pulses):

#try:
    #par['fited_pulses']=True
    Q2 =[]
    tt =[]
    x = []
    ampsa = df9.loc[idx,'stp_mean_fit_bootstrap']
    nbs = int(ampsa.shape[0]/npar)
    nbs=10
    for ibs in range(nbs): #range(nbs):
        
        fig, ax = plt.subplots(figsize=(20, 10))
        par['figure']=fig
        
        par['ibs']=ibs
        #Q4, tt4, x4, algs, colors = fit_TM_cell(idx,df9,par)
        i=idx 




        ########################## 3
        #breakpoint()
        aba_case=False
        if 'aba_case' in par:
            aba_case = par['aba_case']

        model_type='tm5'
        if 'model_type' in par:
            model_type = par['model_type']    

        if par['fited_pulses']==True:
            ampsa = df9.loc[i,'stp_mean_fit_bootstrap']
            siga = df9.loc[i,'stp_sigma_fit_bootstrap']  
            #if abs(ampsa[0])<1e-14:
                #ampsa = df.loc[i,'stp_mean_bootstrap']
                #siga = df.loc[i,'stp_sigma_bootstrap']
        else:
            ampsa = df9.loc[i,'stp_mean_bootstrap']
            siga = df9.loc[i,'stp_sigma_bootstrap']

        Ts = df9.loc[i,'comments1']
        Ts = np.array(Ts[0][:].strip().split()).astype(float)

        #breakpoint()

        #A3 = df.loc[i,'comments']
        #A3i = np.array(A3[0][:].strip().split()).astype(float)
        #A4 = np.ones((len(A3),len(A3i)))
        #for ia3 in range(len(A3)):
        #    A3i = np.array(A3[ia3][:].strip().split()).astype(float)
        #    A4[ia3,:] = A3i
        ##A4.shape 


        ibs  = par['ibs']
        npar = 56
        #amps = ampsa[ibs*npar + np.arange(npar)]
        ampsa1 = ampsa.reshape((-1,npar))
        amps = ampsa1[ibs,:].ravel()

        do_this=1
        if do_this==0:
            #sig  = siga #[(ibs-1)*npar + np.arange(npar)]
            ##A3e  = A4[:,ibs ].ravel()
            siga1 = siga.reshape((-1,npar))
            sig = siga1[ibs,:].ravel()
            
        else:
            #sig = np.std(ampsa1,axis=0).ravel() # reestimate sigma
            sig=pd.DataFrame(ampsa1).std(axis=0).values
        
        DT0 = 25000

        # restrict set of fited data
        vv = par['amplitudes_selected_for_TM_training']
        #vv = np.arange(8)    # A 1:8
        #vv = np.arange(npar) # all
        #vv = np.concatenate([vv,vv+32, vv+44]) # without recovery

        amps0 = amps
        amps = np.zeros(amps.shape)
        amps[vv] = amps0[vv]

             #  tF   p0     tD  dp         A            A1           A2      tDmin  dd   t_FDR t_SMR dp0 
        x0 = [ 100,  0.1,  100, 0.1, amps[0]/0.1, amps[0]/0.1,  amps[0]/0.1, 10,   0.05, 100,  100,  0.02 ]

        x0 = np.array(x0)

        #dxx = np.array([1e3, 20, 1e3, 1e2,  20, 20, 20,  1e1, 20, 1e2 ])
        dxx = np.array([1e3,  10, 1e3, 10,  20, 20, 20,  1e3, 20, 1e3, 1e3, 50 ])

        #breakpoint()
        if model_type=='tm5':
            x0 = x0[0:7]
            dxx = dxx[0:7]

        if model_type=='tm4':
            x0 = x0[0:7]
            dxx = dxx[0:7]

        if model_type=='tm5_fdr2':
            x0 = x0[0:10]
            dxx = dxx[0:10]

        if model_type=='tm5_smr':
            x0 = x0    
            dxx = dxx
        #print(x0)
        #Q0, A0 = QA_TM_aba_synphys(x0, amps, sig, Ts,DT0,model_type)
        ##print(x0)
        #print('initial Q: ',str(Q0))

        #           tF      p0    tD        dp           A                A1                  A2              tDmin   dd   tD2
        #xlower = [  0.1,   0.01,   0.1,     0.0001,     amps[0]/1*1e-1,   amps[0]/1*1e-1,     amps[0]/1*1e-1,  1,     1e-3, 1]
        xlower = x0/dxx
        xlower = np.array(xlower).reshape((-1,))
        #xupper = [  1e5,   1,      1e5,     1,          amps[0]/0.01*1e1, amps[0]/0.01*1e1,   amps[0]/0.01*1e1, 100,  1,    1e4]
        #xupper = np.array(xupper)
        xupper = x0*dxx
        xupper = np.array(xupper).reshape((-1,))
        bounds = [] #np.zeros(len(xlower))
        for ibo in range(len(xlower)):
            bounds = bounds + [(xlower[ibo],xupper[ibo])]
        tt=[]

        x0 = np.array(x0).reshape((-1,))
        
        prob = pg.problem(jit_rosenbrock2(amps.reshape((-1,1)),sig.reshape((-1,1)),Ts,model_type,xlower,xupper))
        pop = pg.population(prob=prob, size = 1000)
        
        t1 = time.time()
        for iii in range(1000):
            prob.fitness(x0) 
        t2 = time.time()
        print(t2 - t1) 
        
        
        uda = pg.nlopt('lbfgs') #'tnewton_precond_restart') #'bobyqa')   #"cobyla")
        algo = pg.algorithm(uda)        
        t1 = time.time()
        pop2 = algo.evolve(pop)
        t2 = time.time()
        x = pop2.champion_x
        x=np.exp(x)
        Q, A = QA_TM_aba_synphys(x, amps, sig, Ts,DT0,model_type)
        plt.plot(np.arange(len(amps)), A, '.-',color='g')
        #print(algs[ii]+ ' Q = '+str(Q)+' '+colors[ii])
        print('n_local_opt: time ' + str(t2 - t1), '; Q ',str(Q))
        
  
        
        uda = pg.de(gen = 100, F = 0.8, CR = 0.9, variant = 2, ftol = 1e-6, xtol = 1e-6)
        algo = pg.algorithm(uda)
        t1 = time.time()
        pop2 = algo.evolve(pop)
        t2 = time.time()
        x = pop2.champion_x
        x=np.exp(x)
        Q, A = QA_TM_aba_synphys(x, amps, sig, Ts,DT0,model_type)
        plt.plot(np.arange(len(amps)), A, '.-',color='c')
        print('differential evolution: time ' + str(t2 - t1), '; Q ',str(Q)) 
        
        
        prob = pg.problem(jit_rosenbrock2(amps.reshape((-1,1)),sig.reshape((-1,1)),Ts,model_type,xlower,xupper))
        #pop = pg.population(prob=prob, size = 1000)
        uda = pg.de1220(gen = 100, allowed_variants = [2,3,7,10,13,14,15,16], variant_adptv = 1, ftol = 1e-6, xtol = 1e-6, memory = False)
        algo = pg.algorithm(uda)
        archi = pg.archipelago(n=32,algo=algo, prob=prob, pop_size=32)
        t1 = time.time()
        #pop2 = algo.evolve(pop)
        archi.evolve() 
        archi.wait()
        t2 = time.time()
        qq=np.array(archi.get_champions_f())
        xx=np.array(archi.get_champions_x())
        
        x = xx[qq.argmin(),:]
        x=np.exp(x)
        Q, A = QA_TM_aba_synphys(x, amps, sig, Ts,DT0,model_type)
        plt.plot(np.arange(len(amps)), A, '.-',color='r')
        print('self adaptive differential evolution: time ' + str(t2 - t1), '; Q ',str(Q)) 
        
        do_this=0
        if do_this==1:
            uda = pg.nsga2(gen = 100, cr = 0.95, eta_c = 10., m = 0.01, eta_m = 50.) 
            algo = pg.algorithm(uda)
            t1 = time.time()
            pop2 = algo.evolve(pop)
            t2 = time.time()
            x = pop2.champion_x
            x=np.exp(x)
            Q, A = QA_TM_aba_synphys(x, amps, sig, Ts,DT0,model_type)
            plt.plot(np.arange(len(amps)), A, '.-',color='m')
            print('Non dominated Sorting Genetic Algorithm (NSGA-II): time ' + str(t2 - t1), '; Q ',str(Q)) 
        
        #algs = ['shgo', 'differential_evolution','dual_annealing','basinhopping',  'random_starts' ]
        #colors = ['y',   'r',                           'c',          'y',         'g' ]
        A3e=1
        plt.errorbar(np.arange(len(amps)), amps*A3e, yerr=sig*A3e, fmt =  'bo')
        

        #algs_short = ['differential_evolution',  'random_starts' ]
        #algs_short = ['n_local_opt' ,'differential_evolution','sade','nsga2','data']
        algs_short = ['n_local_opt' ,'differential_evolution','sade','data']
        #plt.legend(algs)
        plt.legend(algs_short)

        #fig = plt.gcf()
        #set_position(plt.gca, [10, 10, 100, 500], which='both')
        plt.xlim((min(vv)-1,max(vv)+2)) 
        #thismanager = plt.get_current_fig_manager()
        #thismanager.window.SetPosition((500, 0))
        #plt.hold(False)
        plt.pause(0.5) 

In [ ]:
import time
import numpy as np
import pygmo as pg
from numba import jit, float64
dummy_x = np.full((2000,), 1.)

class jit_rosenbrock:
    def fitness(self,x):
        return jit_rosenbrock._fitness(x)
    @jit #(float64[:](float64[:]),nopython=True)
    def _fitness(x):
        retval = np.zeros((1,))
        for i in range(len(x) - 1):
            tmp1 = (x[i + 1]-x[i]*x[i])
            tmp2 = (1.-x[i])
            retval[0] += 100.*tmp1*tmp1+tmp2*tmp2
        return retval
    def get_bounds(self):
        return (np.full((self.dim,),-5.),np.full((self.dim,),10.))
    def __init__(self,dim):
        self.dim = dim
prob_jit = pg.problem(jit_rosenbrock(2000))
start_time = time.time(); [prob_jit.fitness(dummy_x) for i in range(100)]; print(time.time() - start_time) 

In [ ]:
type(prob_jit.fitness(dummy_x))

In [ ]:
from numba import jit, float64
class jit_rosenbrock2:
    def fitness(self,x):
        return jit_rosenbrock2._fitness(x)
    @jit(float64[:](float64[:]),nopython=True)
    def _fitness(x):
        retval = np.zeros((1,))
        for i in range(len(x) - 1):
            tmp1 = (x[i + 1]-x[i]*x[i])
            tmp2 = (1.-x[i])
            retval[0] += 100.*tmp1*tmp1+tmp2*tmp2
        return retval
    def get_bounds(self):
        return (np.full((self.dim,),-5.),np.full((self.dim,),10.))
    def __init__(self,dim):
        self.dim = dim
prob_jit2 = pg.problem(jit_rosenbrock2(2000))
start_time = time.time(); [prob_jit2.fitness(dummy_x) for i in range(1000)]; print(time.time() - start_time) 



### Tsodyks Markram model 5 parameters + 2 parameters to fit 2 phase changing release probability 

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import time
import scipy.optimize as opt
import pygmo as pg
import pandas as pd
#from pylab import *
#class sphere_function:
#    def fitness(self, x):
#        return [sum(x*x)]
#
#     def get_bounds(self):
#        return ([-1,-1],[1,1])

#prob = pg.problem(sphere_function())


t00 = time.time()
Q3 =[]
TT =[]
X = []
algs = ['shgo', 'differential_evolution','dual_annealing','basinhopping',  'random_starts' ]
colors = ['g',   'r',                           'c',          'y',         'b' ]
nalg = len(algs)

npar=56
#vv = np.arange(8)    # A 1:8 20Hz
vv = np.arange(npar) # all
vv = np.delete(vv,np.array([8,12,16,20,24,28,32,40,44,52,56])-1)
#vv = np.concatenate([vv,vv+32, vv+44]) # without recovery

#par = {'fited_pulses':True, 'amplitudes_selected_for_TM_training':vv, 'aba_case':True, 'figure':0,
#       'model_type' : 'tm5','ibs':0 }

par = {'fited_pulses':True, 'amplitudes_selected_for_TM_training':vv, 'aba_case':False, 'figure':0,
       'model_type' : 'tm5','ibs':0, 'dont_plot':0 }

par['model_type'] = 'tm5_smr'  # extended TM model
#par['model_type'] = 'tm5_smr'

for i in  range(len(df9.index)): # #range(39,40): #range(22,23): #range(len(df9.index)):
    idx = df9.index[i]
    print('\n\n'+str(i)+'  '+df9.loc[idx,'comments']+'\n')
    Ts  = df9.loc[idx,'comments1']
    #ampsa = df.loc[idx,'stp_mean_fit_bootstrap']
    #is_fited_pulses = (abs(ampsa[0])>1e-14)
    
    is_fited_pulses = True
    if (len(Ts)!=0)&((par['fited_pulses']==False)|is_fited_pulses):

    #try:
        #par['fited_pulses']=True
        Q2 =[]
        tt =[]
        x = []
        ampsa = df9.loc[idx,'stp_mean_fit_bootstrap']
        nbs = int(ampsa.shape[0]/npar)
        nbs=100
        for ibs in range(nbs):
            fig, ax = plt.subplots(figsize=(20, 10))
            par['figure']=fig
            par['ibs']=ibs
            if ibs>3:
                par['dont_plot']=1
            else:
                par['dont_plot']=0
            Q4, tt4, x4, algs, colors = fit_TM_cell(idx,df9,par)
            nalg = len(tt)
        
            Q2 = Q2 + [Q4]
            tt = tt + [tt4]
            x = x + [x4]
    #except:
    else:
        print(Ts)
        Q2 = (10*np.zeros(nalg)).tolist()
        tt = (0*np.zeros(nalg)).tolist()
        x = (0*np.zeros(nalg)).tolist()
        
    Q3 = Q3 + [Q2]
    TT = TT + [tt]
    X = X + [x]
    

    
#Q3 = pd.DataFrame(Q3)  
#TT = pd.DataFrame(TT) 
t10 = time.time()
print('time : '+ str(t10-t00))


# Save results

#Q3.columns = algs
#algs2=algs
#for ialg, alg in enumerate(algs2): 
#    algs2[ialg] = alg + ' , time'
#TT.columns = algs2
#QT = pd.concat([Q3,TT],axis=1)

#X3 = pd.DataFrame(X)
##X3.columns = ['differential_evolution best x', 'dual_annealing best x', 'random_starts best x', 'shgo best x', 'basinhopping best x']
#X3.columns = ['differential_evolution best x', 'dual_annealing best x', 'random_starts best x']

#QT = pd.concat([QT,X3],axis=1)
#QT.index = df9.index

##QTX = pd.concat([df9,QT],axis=1)
#QTX_extended = pd.concat([df9,QT],axis=1)

#QTX_extended.to_hdf('fit_aba_2019_A1_8_fited_pulses_TM_SMP0_syntypes_results','data')

Xa1=[]
Xa2=[]
Xa3=[]
for i in range(len(X)):
    Xi = np.array(X[i])
    Xa1 = Xa1 +[Xi[:,0,:]]
    Xa2 = Xa2 +[Xi[:,1,:]]
    Xa3 = Xa3 +[Xi[:,2,:]]
    
X3 = pd.DataFrame([Xa1,Xa2,Xa3]).T    
#X3.columns = ['differential_evolution best x', 'dual_annealing best x', 'random_starts best x', 'shgo best x', 'basinhopping best x']
X3.columns = ['differential_evolution best x', 'dual_annealing best x', 'random_starts best x']
X3.index = df9.index

QTX_extended = pd.concat([df9,X3],axis=1)

QTX_extended.to_hdf('fit_aba_2019_mean_fited_pulses_TM_SMP0_100bs_syntypes_results','data')

In [ ]:
QTX_extended.head() 

###  Tsodyks Markram model 5 parameters

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import time
import scipy.optimize as opt
#from pylab import *


t00 = time.time()
Q3 =[]
TT =[]
X = []
algs = ['shgo', 'differential_evolution','dual_annealing','basinhopping',  'random_starts' ]
colors = ['g',   'r',                           'c',          'y',         'b' ]
nalg = len(algs)

npar=56
#vv = np.arange(8)    # A 1:8 20Hz
vv = np.arange(npar) # all
vv = np.delete(vv,np.array([8,12,16,20,24,28,32,40,44,52,56])-1)
#vv = np.concatenate([vv,vv+32, vv+44]) # without recovery
par = {'fited_pulses':True, 'amplitudes_selected_for_TM_training':vv, 'aba_case':True, 'figure':0,
       'model_type' : 'tm5' }

#par['model_type'] = 'TM_5_changing_tD'

for i in  range(len(df9.index)): #range(22,23): #range(len(df9.index)):
    idx = df9.index[i]
    print('\n\n'+str(i)+'  '+df9.loc[idx,'comments']+'\n')
    Ts  = df9.loc[idx,'comments1']
    #ampsa = df.loc[idx,'stp_mean_fit_bootstrap']
    #is_fited_pulses = (abs(ampsa[0])>1e-14)
    
    is_fited_pulses = True
    if (len(Ts)!=0)&((par['fited_pulses']==False)|is_fited_pulses):
        fig, ax = plt.subplots(figsize=(20, 10))
        par['figure']=fig
    #try:
        #par['fited_pulses']=True
        Q2, tt, x, algs, colors = fit_TM_cell(idx,df9,par)
        nalg = len(tt)
    #except:
    else:
        print(Ts)
        Q2 = (10*np.zeros(nalg)).tolist()
        tt = (0*np.zeros(nalg)).tolist()
        x = (0*np.zeros(nalg)).tolist()
        
    Q3 = Q3 + [Q2]
    TT = TT + [tt]
    X = X + [x]
    

    
Q3 = pd.DataFrame(Q3)  
TT = pd.DataFrame(TT) 
t10 = time.time()
print('time : '+ str(t10-t00))

In [ ]:
# Save results

Q3.columns = algs
algs2=algs
for ialg, alg in enumerate(algs2): 
    algs2[ialg] = alg + ' , time'
TT.columns = algs2
QT = pd.concat([Q3,TT],axis=1)

X3 = pd.DataFrame(X)
#X3.columns = ['differential_evolution best x', 'dual_annealing best x', 'random_starts best x', 'shgo best x', 'basinhopping best x']
X3.columns = ['differential_evolution best x', 'dual_annealing best x', 'random_starts best x']

QT = pd.concat([QT,X3],axis=1)
QT.index = df9.index

QTX = pd.concat([df9,QT],axis=1)

QTX.to_hdf('fit_aba_2019_A1_8_fited_pulses_syntypes_results2','data')

In [ ]:
QTX.loc[QTX.index[0],'comments']

In [ ]:
QTX=pd.read_hdf('fit_aba_2019_A1_8_fited_pulses_syntypes_results')

### miscelleneous:

In [ ]:


QTX.shape

In [ ]:
QTX2 = QTX.iloc[0:43,0:14]
QTX3 = QTX.iloc[43:,14:]
QTX3.index = QTX2.index
QTX2 = pd.concat([QTX2, QTX3],axis=1)

In [ ]:
QTX2

In [ ]:
QTX2.to_hdf('fit_aba_2019_A1_8_fited_pulses_syntypes_results2','data')
#QTX2 = QTX2.reset_index()
#QTX2.to_feather('fit_aba_2019_A1_8_fited_pulses_syntypes_results2')

In [ ]:
np.__version__

In [ ]:
import tables
#import numpy as np
#import pandas as pd
tables.__version__

In [ ]:
import importlib
importlib.reload(tables)
tables.__version__

In [ ]:
QTX2=pd.read_hdf('fit_aba_2019_A1_8_fited_pulses_syntypes_results2')
#QTX2=pd.read_feather('fit_aba_2019_A1_8_fited_pulses_syntypes_results2')

In [ ]:
tables.__version__

In [ ]:
import numpy as np
import pandas as pd
QTX2.to_hdf('fit_aba_2019_A1_8_fited_pulses_syntypes_results2','data')
QTX2=pd.read_hdf('fit_aba_2019_A1_8_fited_pulses_syntypes_results2')